<a href="https://colab.research.google.com/github/Jesmeeksingh/Machine_Learning_projects/blob/main/Hinglish_whatsapp_chat_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open('thefreakschat.txt','r') as file:
  hinglish_text = file.read()

In [81]:
# hinglish_text

In [3]:
lines = hinglish_text.split('\n')

In [4]:
messages = []
keywords_to_exclude = ['<Media omitted>','null']
for line in lines:
    message_index = line.find('-') + 2
    message = line[message_index:]
    message_index = message.find(':')
    if message_index != -1:  # Check if ':' is found
        message = message[message_index + 1:]  # Skip the ':'
        if not any(keyword in message for keyword in keywords_to_exclude):
          messages.append(message.strip())


modified_data = '\n'.join(messages)
modified_data

'Koyi Accha Sa Tripod Suggest kro guys..\nAmazon ya Flipkart p ho tohh direct links send krna..\nAaj h lena h\nAaj miloge?\nBakchodi karne\n😋\nTriceps hogyi bkl\n???\nMain nikl rua\nRha*\nAja vai\nCall krdio\n2 hrs — Nurse k sth 🙂🤨\n🌚🌚\nDallo kaha reh gye\n@919311355317\n@919891890270\n@919717575108\nMai panoti ke ghar ke niche hu\nMere ghr aa\nOr piyush banana lene gya hai\nTu iske ghar aa\nBahar chalenge\nAra hu\nDivi bhi aara hai\nBahr kha bc\nLadkia tadenge\nAre bc anmol ke ghr Beth jaate h\nIdhar pankha hai na\nAre yr Beth te h\nAnmol ke ghr Beth te h\nAra hu me\nSunday ko free ho ?\nKya karna hai\nAmrit udyan chlloge?\nhttps://share.mattel163.com/web/deeplink/share?linkid=12892367ef80a3ab\nBhai aaj agr aana ho toh btadena\nTime\nUssi hisab se gym jauinga\n2 -3 transition reel k ideas h..\nMust Watch my Mini Vlog today at 7:30 p.m.\nDivi dalle\nVideo bhej de\n@918527030737\n😋\n@918470084007\nYe vasu ka tujhe bhi aya h mess??\nBadka mujhe lga jay wale topic pe hi start Kiya hai bna

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([modified_data])

In [8]:
tokenizer.word_index

{'hai': 1,
 'h': 2,
 'nhi': 3,
 'bhai': 4,
 'bhi': 5,
 'se': 6,
 'ki': 7,
 'bc': 8,
 'ko': 9,
 'ka': 10,
 'tu': 11,
 'kya': 12,
 'm': 13,
 'ye': 14,
 'to': 15,
 'tha': 16,
 'toh': 17,
 'k': 18,
 'ke': 19,
 'me': 20,
 'na': 21,
 'ho': 22,
 'hi': 23,
 'rha': 24,
 'mein': 25,
 'pe': 26,
 'bkl': 27,
 'this': 28,
 'message': 29,
 'vo': 30,
 'was': 31,
 'abe': 32,
 'or': 33,
 '919717575108': 34,
 'ghar': 35,
 'abhi': 36,
 'aaj': 37,
 'mujhe': 38,
 'hu': 39,
 'main': 40,
 'mere': 41,
 'aa': 42,
 'vai': 43,
 'ab': 44,
 'ni': 45,
 'koi': 46,
 '919311355317': 47,
 'liye': 48,
 'tohh': 49,
 'mei': 50,
 'ghr': 51,
 '918470084007': 52,
 'gand': 53,
 'anmol': 54,
 'haa': 55,
 '919310951714': 56,
 'gya': 57,
 'ya': 58,
 'kar': 59,
 'hmm': 60,
 'mai': 61,
 'kr': 62,
 'mera': 63,
 'kal': 64,
 'le': 65,
 'edited': 66,
 'bje': 67,
 'ok': 68,
 'hoga': 69,
 '919891890270': 70,
 'ek': 71,
 'tere': 72,
 'tujhe': 73,
 'time': 74,
 'tum': 75,
 '😋': 76,
 'chl': 77,
 'bta': 78,
 'ha': 79,
 'de': 80,
 'aur': 81,


In [9]:
input_sequences = []
for sentence in modified_data.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
     input_sequences.append(tokenized_sentence[:i+1])

In [10]:
input_sequences

[[210, 164],
 [210, 164, 539],
 [210, 164, 539, 2622],
 [210, 164, 539, 2622, 3165],
 [210, 164, 539, 2622, 3165, 344],
 [210, 164, 539, 2622, 3165, 344, 167],
 [2623, 58],
 [2623, 58, 1303],
 [2623, 58, 1303, 136],
 [2623, 58, 1303, 136, 22],
 [2623, 58, 1303, 136, 22, 49],
 [2623, 58, 1303, 136, 22, 49, 801],
 [2623, 58, 1303, 136, 22, 49, 801, 5508],
 [2623, 58, 1303, 136, 22, 49, 801, 5508, 709],
 [2623, 58, 1303, 136, 22, 49, 801, 5508, 709, 320],
 [37, 2],
 [37, 2, 540],
 [37, 2, 540, 2],
 [37, 454],
 [158, 193],
 [1189, 575],
 [1189, 575, 27],
 [40, 333],
 [40, 333, 5509],
 [291, 43],
 [123, 1304],
 [95, 3166],
 [95, 3166, 2287],
 [95, 3166, 2287, 5510],
 [95, 3166, 2287, 5510, 18],
 [95, 3166, 2287, 5510, 18, 1190],
 [95, 3166, 2287, 5510, 18, 1190, 5511],
 [710, 126],
 [710, 126, 689],
 [710, 126, 689, 194],
 [61, 443],
 [61, 443, 19],
 [61, 443, 19, 35],
 [61, 443, 19, 35, 19],
 [61, 443, 19, 35, 19, 914],
 [61, 443, 19, 35, 19, 914, 39],
 [41, 51],
 [41, 51, 42],
 [33, 161],

In [11]:
max_len = max([len(x) for x in input_sequences])
max_len

118

In [12]:
#zero padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
padded_input_sequences = pad_sequences(input_sequences,maxlen = max_len,padding = 'pre')

In [14]:
X = padded_input_sequences[:,:-1]

In [15]:
X

array([[    0,     0,     0, ...,     0,     0,   210],
       [    0,     0,     0, ...,     0,   210,   164],
       [    0,     0,     0, ...,   210,   164,   539],
       ...,
       [    0,     0,     0, ...,    15, 12111,     7],
       [    0,     0,     0, ...,     0,     0,  1218],
       [    0,     0,     0, ...,     0,  1218, 12113]], dtype=int32)

In [16]:
X.shape

(90815, 117)

In [17]:
Y = padded_input_sequences[:,-1]

In [23]:
Y

array([  164,   539,  2622, ..., 12112, 12113,   319], dtype=int32)

In [18]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y,num_classes=12114)

In [19]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
Y.shape

(90815, 12114)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [22]:
model = Sequential()
model.add(Embedding(12114,100,input_length=117))
model.add(LSTM(150))
model.add(Dense(12114,activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 117, 100)          1211400   
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 12114)             1829214   
                                                                 
Total params: 3191214 (12.17 MB)
Trainable params: 3191214 (12.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.fit(X,Y,epochs=30)

Epoch 1/30
2838/2838 [==============================] - 64s 21ms/step - loss: 7.3314 - accuracy: 0.0444
Epoch 2/30
2838/2838 [==============================] - 40s 14ms/step - loss: 6.6229 - accuracy: 0.0809
Epoch 3/30
2838/2838 [==============================] - 39s 14ms/step - loss: 6.1136 - accuracy: 0.1099
Epoch 4/30
2838/2838 [==============================] - 38s 14ms/step - loss: 5.6572 - accuracy: 0.1321
Epoch 5/30
2838/2838 [==============================] - 39s 14ms/step - loss: 5.2269 - accuracy: 0.1548
Epoch 6/30
2838/2838 [==============================] - 38s 13ms/step - loss: 4.8137 - accuracy: 0.1827
Epoch 7/30
2838/2838 [==============================] - 38s 13ms/step - loss: 4.4283 - accuracy: 0.2165
Epoch 8/30
2838/2838 [==============================] - 38s 14ms/step - loss: 4.0716 - accuracy: 0.2583
Epoch 9/30
2838/2838 [==============================] - 37s 13ms/step - loss: 3.7514 - accuracy: 0.3027
Epoch 10/30
2838/2838 [==============================] - 37s 13m

In [80]:
import numpy as np
text = "tanvir ko"
for i in range(5):

  token_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([token_text],maxlen = 117,padding='pre')
  pos = np.argmax(model.predict(padded_token_text))
  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)


1/1 [==============================] - 0s 20ms/step
tanvir ko bhi
1/1 [==============================] - 0s 22ms/step
tanvir ko bhi utha
1/1 [==============================] - 0s 19ms/step
tanvir ko bhi utha lenge
1/1 [==============================] - 0s 21ms/step
tanvir ko bhi utha lenge ab
1/1 [==============================] - 0s 19ms/step
tanvir ko bhi utha lenge ab ab
